# Базы данных. Часть 2

## Базы данных и Python

Питон позволяет работать с различными СУБД, но проще всего использовать встроенную библиотеку [SQLite](https://docs.python.org/3.5/library/sqlite3.html). Важно также запомнить, что файлы баз данных имеют расширение `.db`.

In [2]:
import sqlite3

# подключаемся к базе данных
conn = sqlite3.connect('example2.db')

# создаем объект "курсор", которому будем передавать запросы
c = conn.cursor()

# создаем таблицу
c.execute("CREATE TABLE IF NOT EXISTS students(name text, major text, year integer)")

# вставляем строку
c.execute("INSERT INTO students VALUES ('Петя','филология',1), ('Настя','лингвистика',4)")

# сохраняем изменения
conn.commit()

# отключаемся от БД
#conn.close()

**Важная вещь №1**: после подключения к БД нужно обязательно инициализировать курсор, иначе вы не сможете делать к ней запросы!

**Важная вещь №2**: если вы что-то изменили, нужно обязательно закоммитить изменения, иначе они не сохранятся в файле БД!

И немного о безопасности: при создании запроса нельзя использовать конкатенацию строк и форматирование строк, как в питоне. Это сделает ваше приложение уязвимым для SQL-инъекций - особых хакерских атак, которые заключаются в подставлении в запрос нежелательных комманд - например, DROP TABLE. Поподробнее об этом можно почитать [вот здесь](https://habrahabr.ru/post/148151/).

![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

In [3]:
# Так нельзя!
name = 'Петя'
c.execute("SELECT * FROM students WHERE name = '%s'" % name)

# Вот как надо
x = ('Петя',)
c.execute('SELECT * FROM students WHERE name=?', x)
print(c.fetchone())

('Петя', 'филология', 1)


In [4]:
# Если результатом запроса является несколько строк, можно по ним итерировать

for row in c.execute('SELECT * FROM students ORDER BY year'):
    print(row)

('Петя', 'филология', 1)
('Настя', 'лингвистика', 4)


In [5]:
# как подставить несколько переменных в sql-запрос

x = 'Аня'
y = 'математика'
z = 3

c.execute('INSERT INTO students VALUES (?, ?, ?)', (x, y, z))
conn.commit()

#### Форматирование строк

Если нужно подставить переменные в качестве названия таблицы или колонок, то придется использовать форматирование строк.

In [6]:
params = ['vowel', 'f1', 'f2']
c.execute('CREATE TABLE vowels({}, {}, {})'.format(params[0], params[1], params[2]))

In [7]:
# как написать длинный запрос посимпатичнее
c.execute('''
INSERT INTO vowels 
VALUES 
('a', 1234.56, 4567.8), 
('u', 1111.1, 3333.3)'''
)

In [8]:
for row in c.execute('SELECT * FROM vowels'):
    print(row)

('a', 1234.56, 4567.8)
('u', 1111.1, 3333.3)


#### Функции курсора

* **fetchone()** -- возвращает следующий элемент из результата запроса (т.е. одну строку из бд). Результат -- кортеж, где элементом является значение каждой из колонок или None
* **fetchall()** -- возвращает все результаты запроса в виде списка
* **fetchmany()** -- взвращает заданное количество строк из результатов запроса

In [9]:
# извлекаем строки по одной
# обратите внимание, что после каждого вызова fetchone возвращает следующую строку!
c.execute('SELECT * FROM students ORDER BY year')
print(c.fetchone())
print(c.fetchone())
print(c.fetchone())

('Петя', 'филология', 1)
('Аня', 'математика', 3)
('Настя', 'лингвистика', 4)


In [10]:
# извлекаем две строки
c.execute('SELECT * FROM students ORDER BY year')
print(c.fetchmany(2))

[('Петя', 'филология', 1), ('Аня', 'математика', 3)]


In [11]:
# извлекаем все строки
c.execute('SELECT * FROM students ORDER BY year')
print(c.fetchall())

[('Петя', 'филология', 1), ('Аня', 'математика', 3), ('Настя', 'лингвистика', 4)]


# Практика #2